In [56]:
import re
import requests
from bs4 import BeautifulSoup
from pprint import pprint

url = 'https://www.melon.com/chart/index.htm'

headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'
}

res = requests.get(url, headers=headers)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')

    atag_list = soup.select("a[href*='playSong']")

    song_list = []

    for idx, atag in enumerate(atag_list, 1):
        song_dict = {}

        title = atag.text
        song_dict['title'] = title
        href = atag['href']

        matched = re.search(r'(\d+)\)', href)
        
        if matched:
            song_id = matched.group(1)
        song_dict['id'] = song_id

        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        song_dict['url'] = song_url

        song_list.append(song_dict)
        
    pprint(len(song_list))
    pprint(song_list)

else:
    print(f'Error Code = {res.status_code}')

100
[{'id': '38589554',
  'title': 'TOO BAD (feat. Anderson .Paak)',
  'url': 'https://www.melon.com/song/detail.htm?songId=38589554'},
 {'id': '38429074',
  'title': '모르시나요(PROD.로코베리)',
  'url': 'https://www.melon.com/song/detail.htm?songId=38429074'},
 {'id': '38629386',
  'title': 'like JENNIE',
  'url': 'https://www.melon.com/song/detail.htm?songId=38629386'},
 {'id': '36397952',
  'title': 'Drowning',
  'url': 'https://www.melon.com/song/detail.htm?songId=36397952'},
 {'id': '38242510',
  'title': 'HOME SWEET HOME (feat. 태양, 대성)',
  'url': 'https://www.melon.com/song/detail.htm?songId=38242510'},
 {'id': '38123338',
  'title': '나는 반딧불',
  'url': 'https://www.melon.com/song/detail.htm?songId=38123338'},
 {'id': '38444825',
  'title': 'REBEL HEART',
  'url': 'https://www.melon.com/song/detail.htm?songId=38444825'},
 {'id': '38123332',
  'title': 'Whiplash',
  'url': 'https://www.melon.com/song/detail.htm?songId=38123332'},
 {'id': '38426197',
  'title': '오늘만 I LOVE YOU',
  'url': 'h

In [59]:
import re
import requests
from bs4 import BeautifulSoup

headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

# Song 100곡의 상세정보 목록을 저장할 list 선언
song_lyric_list = list()
print('===> 100 곡 노래 파싱 시작')
for idx,song in enumerate(song_list,1):
    print(f'==> {idx} {song['title']}')
    # Song 1곡의 상세정보를 저장할 dict 선언
    song_lyric_dict = dict()
    
    res = requests.get(song['url'], headers=headers)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        song_lyric_dict['곡명'] = song['title']
        
        singer_span = soup.select_one("a[href*='goArtistDetail'] span")
        song_lyric_dict['가수'] = singer_span.text
        
        song_dd = soup.select('div.meta dd') #song_dd는 ResultSet타입, song_dd[0]는 Tag 타입입
        if song_dd:
            song_lyric_dict['앨범'] = song_dd[0].text
            song_lyric_dict['발매일'] = song_dd[1].text
            song_lyric_dict['장르'] = song_dd[2].text
        
        # Song 상세정보 링크
        song_lyric_dict['detail_url'] = song['url']
            
        # 좋아요 건수
        song_id = song['id']
        ajax_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        res = requests.get(ajax_url, headers=headers)
        if res.ok:
            song_lyric_dict['좋아요'] = res.json()['contsLike'][0]['SUMMCNT']
        
        # 노래 가사     
        lyric_div = soup.select('div#d_video_summary')
        if lyric_div:
            lyric = lyric_div[0].text
        else:
            lyric = ''            
        
        # \n\r\t 특수문자를 찾는 Pattern객체 생성
        pattern = re.compile(r'[\n\r\t]')
        song_lyric_dict['가사'] = pattern.sub('', lyric)
        
        # list에 상세정보 담은 dict를 저장
        song_lyric_list.append(song_lyric_dict)
    else:
        print(f'Error Code = {res.status_code}')
        
print(len(song_lyric_list))
pprint(song_lyric_list[:2])
print('===> 100 곡 노래 파싱 끝')

===> 100 곡 노래 파싱 시작
==> 1 TOO BAD (feat. Anderson .Paak)
==> 2 모르시나요(PROD.로코베리)
==> 3 like JENNIE
==> 4 Drowning
==> 5 HOME SWEET HOME (feat. 태양, 대성)
==> 6 나는 반딧불
==> 7 REBEL HEART
==> 8 Whiplash
==> 9 오늘만 I LOVE YOU
==> 10 APT.
==> 11 HOT
==> 12 HAPPY
==> 13 ATTITUDE
==> 14 toxic till the end
==> 15 내게 사랑이 뭐냐고 물어본다면
==> 16 I DO ME
==> 17 Flower
==> 18 PO￦ER
==> 19 TAKE ME
==> 20 미치게 그리워서
==> 21 소나기
==> 22 Dash
==> 23 한 페이지가 될 수 있게
==> 24 사랑은 늘 도망가
==> 25 천상연


KeyboardInterrupt: 

In [61]:
# [{'가수';'BTS','앨범':''},{}]
import pandas as pd

#컬럼명을 설정하면서 empty DataFrame 객체생성
song_list_df = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','detail_url','좋아요','가사'])
for song_lyric in song_lyric_list: #[ {},{},{} ]
    df_new_row = pd.DataFrame.from_records([song_lyric])
    song_list_df = pd.concat([song_list_df, df_new_row])
    
song_list_df.tail(3)

,곡명,가수,앨범,발매일,장르,detail_url,좋아요,가사
0,Dash,PLAVE,Caligo Pt.1,2025.02.03,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,53568,실낱같았던한 줄기의 빛을 봤어칠흑 같은 어둠 속에끝을 헤매고 있어아름답던 영원처럼아...
0,한 페이지가 될 수 있게,DAY6 (데이식스),The Book of Us : Gravity,2019.07.15,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,348533,솔직히 말할게많이 기다려 왔어너도 그랬을 거라 믿어오늘이 오길매일같이 달력을 보면서...
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",https://www.melon.com/song/detail.htm?songId=3...,225173,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...


In [67]:
import json

with open('songs100.json','w',encoding='utf-8') as file:
    json.dump(song_lyric_list, file)

In [6]:
import pandas as pd

song_df = pd.read_json('songs100.json')
print(type(song_df))
song_df.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,detail_url,좋아요,가사
0,TOO BAD (feat. Anderson .Paak),G-DRAGON,Übermensch,2025.02.25,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,124477,"‘G’, ‘A.P’“Let me kill ’em like I usually do, ..."
1,모르시나요(PROD.로코베리),조째즈,모르시나요,2025.01.07,발라드,https://www.melon.com/song/detail.htm?songId=3...,58386,찬바람 불어오니그대 생각에 눈물짓네인사 없이 떠나시던 날그리움만 남겨놓고그리워 글썽...
2,like JENNIE,제니 (JENNIE),Ruby,2025.03.07,댄스,https://www.melon.com/song/detail.htm?songId=3...,54416,"Come on, it’s gon be f hardSpecial edition and..."
3,Drowning,WOODZ,OO-LI,2023.04.26,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,167698,미치도록 사랑했던지겹도록 다투었던네가 먼저 떠나고여긴 온종일 비가 왔어금세 턱 끝까...
4,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON,"HOME SWEET HOME (feat. 태양, 대성)",2024.11.22,랩/힙합,https://www.melon.com/song/detail.htm?songId=3...,206005,"You say, It’s changedShow must go on, Behave오랜..."


In [70]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,detail_url,좋아요,가사
19,미치게 그리워서,황가람,미치게 그리워서,2025.02.09,발라드,https://www.melon.com/song/detail.htm?songId=3...,21439,땅거미진 거리에 어둠이 잦아들면저 거리 불빛 가슴을 친다찬 바람에 무뎌진 사소한 두...
20,소나기,이클립스 (ECLIPSE),선재 업고 튀어 OST Part 1,2024.04.08,"발라드, 국내드라마",https://www.melon.com/song/detail.htm?songId=3...,177677,그치지 않기를 바랬죠처음 그대 내게로 오던 그날에잠시 동안 적시는그런 비가 아니길간...
21,Dash,PLAVE,Caligo Pt.1,2025.02.03,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,53568,실낱같았던한 줄기의 빛을 봤어칠흑 같은 어둠 속에끝을 헤매고 있어아름답던 영원처럼아...
22,한 페이지가 될 수 있게,DAY6 (데이식스),The Book of Us : Gravity,2019.07.15,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,348533,솔직히 말할게많이 기다려 왔어너도 그랬을 거라 믿어오늘이 오길매일같이 달력을 보면서...
23,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",https://www.melon.com/song/detail.htm?songId=3...,225173,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...


In [71]:
# 가수 별 Row Counting
print(type(song_df['가수']))
song_df['가수'].value_counts().head()

<class 'pandas.core.series.Series'>


가수
G-DRAGON       4
황가람            2
DAY6 (데이식스)    2
IVE (아이브)      2
로제 (ROSÉ)      2
Name: count, dtype: int64

In [72]:
# 장르 별 Row Counting
song_df['장르'].value_counts()

장르
댄스            9
록/메탈          5
발라드           4
랩/힙합          3
발라드, 국내드라마    2
발라드, 인디음악     1
Name: count, dtype: int64

In [73]:
# 특정 가수의 노래 정보 출력하기
#song_df.loc[]
song_df['가수'] == 'G-DRAGON'

0      True
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18     True
19    False
20    False
21    False
22    False
23    False
Name: 가수, dtype: bool

In [74]:
song_df.loc[song_df['가수'] == "G-DRAGON", "곡명" : "장르"]

,곡명,가수,앨범,발매일,장르
0,TOO BAD (feat. Anderson .Paak),G-DRAGON,Übermensch,2025.02.25,랩/힙합
4,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON,"HOME SWEET HOME (feat. 태양, 대성)",2024.11.22,랩/힙합
17,PO￦ER,G-DRAGON,PO￦ER,2024.10.31,랩/힙합
18,TAKE ME,G-DRAGON,Übermensch,2025.02.25,댄스


In [75]:
song_df['가수'].unique()

array(['G-DRAGON', '조째즈', '제니 (JENNIE)', 'WOODZ', '황가람', 'IVE (아이브)',
       'aespa', 'BOYNEXTDOOR', '로제 (ROSÉ)', 'LE SSERAFIM (르세라핌)',
       'DAY6 (데이식스)', '로이킴', 'KiiiKiii (키키)', '오반(OVAN)',
       '이클립스 (ECLIPSE)', 'PLAVE', '임영웅'], dtype=object)

In [80]:
song_df['앨범'].sample(10)

23               신사와 아가씨 OST Part.2
10                              HOT
19                         미치게 그리워서
5                            나는 반딧불
6                       IVE EMPATHY
20              선재 업고 튀어 OST Part 1
9                              APT.
7     Whiplash - The 5th Mini Album
14                 내게 사랑이 뭐냐고 물어본다면
17                            PO￦ER
Name: 앨범, dtype: object

In [91]:
print(type(song_df['앨범'].str))
song_df.loc[song_df['앨범'].str.contains('OST')]

<class 'pandas.core.strings.accessor.StringMethods'>


,곡명,가수,앨범,발매일,장르,detail_url,좋아요,가사
20,소나기,이클립스 (ECLIPSE),선재 업고 튀어 OST Part 1,2024.04.08,"발라드, 국내드라마",https://www.melon.com/song/detail.htm?songId=3...,177677,그치지 않기를 바랬죠처음 그대 내게로 오던 그날에잠시 동안 적시는그런 비가 아니길간...
23,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",https://www.melon.com/song/detail.htm?songId=3...,225173,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...


In [8]:
song_df.loc[song_df['좋아요'] == song_df['좋아요'].max()]

,곡명,가수,앨범,발매일,장르,detail_url,좋아요,가사
22,한 페이지가 될 수 있게,DAY6 (데이식스),The Book of Us : Gravity,2019.07.15,록/메탈,https://www.melon.com/song/detail.htm?songId=3...,348533,솔직히 말할게많이 기다려 왔어너도 그랬을 거라 믿어오늘이 오길매일같이 달력을 보면서...


In [7]:
mean_like_value = song_df['좋아요'].mean()
print(mean_like_value)

song_df.loc[song_df['좋아요'] >= mean_like_value,'곡명':'장르'].reset_index(drop=True)

118169.875


,곡명,가수,앨범,발매일,장르
0,TOO BAD (feat. Anderson .Paak),G-DRAGON,Übermensch,2025.02.25,랩/힙합
1,Drowning,WOODZ,OO-LI,2023.04.26,록/메탈
2,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON,"HOME SWEET HOME (feat. 태양, 대성)",2024.11.22,랩/힙합
3,나는 반딧불,황가람,나는 반딧불,2024.10.21,발라드
4,Whiplash,aespa,Whiplash - The 5th Mini Album,2024.10.21,댄스
5,오늘만 I LOVE YOU,BOYNEXTDOOR,오늘만 I LOVE YOU,2025.01.06,댄스
6,APT.,로제 (ROSÉ),APT.,2024.10.18,댄스
7,HAPPY,DAY6 (데이식스),Fourever,2024.03.18,록/메탈
8,PO￦ER,G-DRAGON,PO￦ER,2024.10.31,랩/힙합
9,소나기,이클립스 (ECLIPSE),선재 업고 튀어 OST Part 1,2024.04.08,"발라드, 국내드라마"
